In [1]:
from datetime import datetime, timedelta
import requests
import pandas
import time
today = datetime.now()
edt = time.mktime(today.timetuple())
sdt = time.mktime((today - timedelta(days = 360)).timetuple())
api_url = f'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start={sdt}&end={edt}&period=86400'
res = requests.get(api_url)
df = pandas.DataFrame(res.json())
df['date'] = pandas.to_datetime(df['date'], unit = 's')
df.head()

,date,high,low,open,close,volume,quoteVolume,weightedAverage
0,2020-02-01 01:13:35,9385.381965,9385.381965,9385.381965,9385.381965,0.000000e+00,0.000000,9385.381965
1,2020-02-02 00:00:00,9475.000000,9163.600000,9385.381965,9334.571065,3.054696e+07,3258.636775,9374.151820
2,2020-02-03 00:00:00,9600.000000,9236.905812,9334.202358,9296.350031,2.535231e+07,2713.973708,9341.397871
3,2020-02-04 00:00:00,9360.543825,9092.909472,9295.675015,9194.558171,3.420635e+07,3710.230240,9219.469318
4,2020-02-05 00:00:00,9739.254693,9177.100000,9194.614905,9609.607399,3.753042e+07,3973.576179,9444.998895


## 安裝 Backtesting 

In [2]:
! pip install backtesting

In [3]:
df.columns = ['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'QuoteVolume',
       'WeightedAverage']

In [4]:
df.head()

,Date,High,Low,Open,Close,Volume,QuoteVolume,WeightedAverage
0,2020-02-01 01:13:35,9385.381965,9385.381965,9385.381965,9385.381965,0.000000e+00,0.000000,9385.381965
1,2020-02-02 00:00:00,9475.000000,9163.600000,9385.381965,9334.571065,3.054696e+07,3258.636775,9374.151820
2,2020-02-03 00:00:00,9600.000000,9236.905812,9334.202358,9296.350031,2.535231e+07,2713.973708,9341.397871
3,2020-02-04 00:00:00,9360.543825,9092.909472,9295.675015,9194.558171,3.420635e+07,3710.230240,9219.469318
4,2020-02-05 00:00:00,9739.254693,9177.100000,9194.614905,9609.607399,3.753042e+07,3973.576179,9444.998895


In [5]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 5)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(df, SmaCross, commission=.002, cash = 10000,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


In [6]:
stats

Start                                       0
End                                       360
Duration                                  360
Exposure Time [%]                     73.1302
Equity Final [$]                      17805.7
Equity Peak [$]                       20755.1
Return [%]                            78.0568
Buy & Hold Return [%]                  244.23
Return (Ann.) [%]                         NaN
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                    -22.9235
Avg. Drawdown [%]                    -4.60958
Max. Drawdown Duration                    119
Avg. Drawdown Duration                13.9048
# Trades                                   14
Win Rate [%]                          35.7143
Best Trade [%]                        64.7323
Worst Trade [%]                      -11.4324
Avg. Trade [%]                    

In [7]:
class smacross_without_short(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 5)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            if len(self.trades) > 0:
                self.trades[0].close()


bt = Backtest(df, smacross_without_short, cash = 10000,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  app.launch_new_instance()
/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  app.launch_new_instance()


In [8]:
import talib 

class KDCross(Strategy):
    def init(self):
        
        self.slowk, self.slowd = self.I(talib.STOCH, self.data.High,self.data.Low,
                                            self.data.Close, fastk_period = 9,slowk_period=3,slowd_period=3)

    def next(self):
        if crossover(self.slowk, self.slowd):
            self.buy()
        elif crossover(self.slowd, self.slowk):
            self.sell()


bt = Backtest(df, KDCross, cash = 10000, exclusive_orders=True)
stats = bt.run()
bt.plot()

/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  app.launch_new_instance()
/Users/davidchiu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  app.launch_new_instance()
